In [31]:
E=210e6 #kN/m^2
mu=0.3
t=0.025 #m
w=3000 #kN/m^2

In [48]:
import numpy as np
#STEP1 Discretizing
def LinearTriangleElementStiffness(E,mu,t,xi,yi,xj,yj,xm,ym,p):
    A = (xi*(yj-ym)+ xj*(ym-yi) + xm*(yi-yj))/2
    betai = yj-ym
    betaj = ym-yi
    betam = yi-yj
    gammai = xm-xj
    gammaj = xi-xm
    gammam = xj-xi
    B=np.asarray([[betai, 0 ,betaj, 0, betam, 0],[0, gammai, 0, gammaj, 0 ,gammam],[gammai,betai,gammaj,betaj,gammam,betam]])

    B=B*(1/(2*A))
    if p==1:
        D=(E/(1-mu*mu))*np.asarray([[1,mu,0],[mu,1,0],[0,0,(1-mu)/2]])
    elif p==2:
        D=(E/((1+mu)*(1-2*mu)))*np.asarray([[1-mu,mu,0],[mu,1-mu,0],[0,0,(1-2*mu)/2]])
    
    return t*A*np.dot(B.T,np.dot(D,B))
    



In [50]:
k1=LinearTriangleElementStiffness(E,mu,t,0,0,0.5,0.25,0,0.25,1)
print(k1)

[[ 2019230.76923077        0.                0.         -1009615.38461538
  -2019230.76923077  1009615.38461538]
 [       0.          5769230.76923077  -865384.61538462        0.
    865384.61538462 -5769230.76923077]
 [       0.          -865384.61538462  1442307.69230769        0.
  -1442307.69230769   865384.61538462]
 [-1009615.38461538        0.                0.           504807.69230769
   1009615.38461538  -504807.69230769]
 [-2019230.76923077   865384.61538462 -1442307.69230769  1009615.38461538
   3461538.46153846 -1875000.        ]
 [ 1009615.38461538 -5769230.76923077   865384.61538462  -504807.69230769
  -1875000.          6274038.46153846]]


In [51]:
k2=LinearTriangleElementStiffness(E,mu,t,0,0,0.5,0,0.5,0.25,1)
print(k2)

[[ 1442307.69230769        0.         -1442307.69230769   865384.61538462
         0.          -865384.61538462]
 [       0.           504807.69230769  1009615.38461538  -504807.69230769
  -1009615.38461538        0.        ]
 [-1442307.69230769  1009615.38461538  3461538.46153846 -1875000.
  -2019230.76923077   865384.61538462]
 [  865384.61538462  -504807.69230769 -1875000.          6274038.46153846
   1009615.38461538 -5769230.76923077]
 [       0.         -1009615.38461538 -2019230.76923077  1009615.38461538
   2019230.76923077        0.        ]
 [ -865384.61538462        0.           865384.61538462 -5769230.76923077
         0.          5769230.76923077]]


In [52]:
#STEP 2 Assembling
K = np.zeros((8, 8))


def LinearTriangleAssemble(K :np.ndarray, k, i, j, m):
    K[2*i-1-1, 2*i-1-1] = K[2*i-1-1, 2*i-1-1] + k[1-1, 1-1]
    K[2*i-1-1, 2*i-1] = K[2*i-1-1,  2*i-1] + k[1-1, 2-1]
    K[2*i-1-1, 2*j-1-1] = K[2*i-1-1,  2*j-1-1] + k[1-1, 3-1]
    K[2*i-1-1,  2*j-1] = K[2*i-1-1,  2*j-1] + k[1-1, 4-1]
    K[2*i-1-1,  2*m-1-1] = K[2*i-1-1,  2*m-1-1] + k[1-1, 5-1]
    K[2*i-1-1,  2*m-1] = K[2*i-1-1,  2*m-1] + k[1-1, 6-1]
    K[2*i-1, 2*i-1-1] = K[2*i-1, 2*i-1-1] + k[2-1, 1-1]
    K[2*i-1, 2*i-1] = K[2*i-1, 2*i-1] + k[2-1, 2-1]
    K[2*i-1, 2*j-1-1] = K[2*i-1, 2*j-1-1] + k[2-1, 3-1]
    K[2*i-1, 2*j-1] = K[2*i-1, 2*j-1] + k[2-1, 4-1]
    K[2*i-1, 2*m-1-1] = K[2*i-1, 2*m-1-1] + k[2-1, 5-1]
    K[2*i-1, 2*m-1] = K[2*i-1, 2*m-1] + k[2-1, 6-1]
    K[2*j-1-1, 2*i-1-1] = K[2*j-1-1,  2*i-1-1] + k[3-1, 1-1]
    K[2*j-1-1,  2*i-1] = K[2*j-1-1,  2*i-1] + k[3-1, 2-1]
    K[2*j-1-1,  2*j-1-1] = K[2*j-1-1,  2*j-1-1] + k[3-1, 3-1]
    K[2*j-1-1,  2*j-1] = K[2*j-1-1,  2*j-1] + k[3-1, 4-1]
    K[2*j-1-1,  2*m-1-1] = K[2*j-1-1,  2*m-1-1] + k[3-1, 5-1]
    K[2*j-1-1,  2*m-1] = K[2*j-1-1, 2*m-1] + k[3-1, 6-1]
    K[2*j-1, 2*i-1-1] = K[2*j-1, 2*i-1-1] + k[4-1, 1-1]
    K[2*j-1, 2*i-1] = K[2*j-1, 2*i-1] + k[4-1, 2-1]
    K[2*j-1, 2*j-1-1] = K[2*j-1, 2*j-1-1] + k[4-1, 3-1]
    K[2*j-1, 2*j-1] = K[2*j-1, 2*j-1] + k[4-1, 4-1]
    K[2*j-1, 2*m-1-1] = K[2*j-1, 2*m-1-1] + k[4-1, 5-1]
    K[2*j-1, 2*m-1] = K[2*j-1, 2*m-1] + k[4-1, 6-1]
    K[2*m-1-1,  2*i-1-1] = K[2*m-1-1,  2*i-1-1] + k[5-1, 1-1]
    K[2*m-1-1,  2*i-1] = K[2*m-1-1, 2*i-1] + k[5-1, 2-1]
    K[2*m-1-1,  2*j-1-1] = K[2*m-1-1,  2*j-1-1] + k[5-1, 3-1]
    K[2*m-1-1,  2*j-1] = K[2*m-1-1,  2*j-1] + k[5-1, 4-1]
    K[2*m-1-1,  2*m-1-1] = K[2*m-1-1,  2*m-1-1] + k[5-1, 5-1]
    K[2*m-1-1, 2*m-1] = K[2*m-1-1,  2*m-1] + k[5-1, 6-1]
    K[2*m-1, 2*i-1-1] = K[2*m-1, 2*i-1-1] + k[6-1, 1-1]
    K[2*m-1, 2*i-1] = K[2*m-1, 2*i-1] + k[6-1, 2-1]
    K[2*m-1, 2*j-1-1] = K[2*m-1, 2*j-1-1] + k[6-1, 3-1]
    K[2*m-1, 2*j-1] = K[2*m-1, 2*j-1] + k[6-1, 4-1]
    K[2*m-1, 2*m-1-1] = K[2*m-1, 2*m-1-1] + k[6-1, 5-1]
    K[2*m-1, 2*m-1] = K[2*m-1, 2*m-1] + k[6-1, 6-1]
    
    return K
    

In [53]:
K = np.zeros((8, 8))
K=LinearTriangleAssemble(K,k1,1,3,4)
K=LinearTriangleAssemble(K,k2,1,2,3)

print(K)

[[ 3461538.46153846        0.         -1442307.69230769   865384.61538462
         0.         -1875000.         -2019230.76923077  1009615.38461538]
 [       0.          6274038.46153846  1009615.38461538  -504807.69230769
  -1875000.                0.           865384.61538462 -5769230.76923077]
 [-1442307.69230769  1009615.38461538  3461538.46153846 -1875000.
  -2019230.76923077   865384.61538462        0.                0.        ]
 [  865384.61538462  -504807.69230769 -1875000.          6274038.46153846
   1009615.38461538 -5769230.76923077        0.                0.        ]
 [       0.         -1875000.         -2019230.76923077  1009615.38461538
   3461538.46153846        0.         -1442307.69230769   865384.61538462]
 [-1875000.                0.           865384.61538462 -5769230.76923077
         0.          6274038.46153846  1009615.38461538  -504807.69230769]
 [-2019230.76923077   865384.61538462        0.                0.
  -1442307.69230769  1009615.38461538  3461538.4

In [54]:
#STEP 4 Applying Bpundary conditions
U=np.zeros((8,1))
F=np.zeros((8,1))

#COnditions
F[2,0]=9.375
F[3,0]=0
F[4,0]=9.375
F[5,0]=0

#Partitioning matrices
Up=U[2:6]
Ux=U[[0,1,6,7],:]
Fp=F[2:6]
Kpp=K[2:6,2:6]
print(Kpp)
print(Fp)
Up=np.dot(np.linalg.inv(Kpp),Fp)

print(Up)
# Fx=np.add(np.dot(Kxp,Up),np.dot(Kxx,Ux))
# U=np.concatenate((Up,Ux))
# F=np.concatenate((Fp,Fx))



[[ 3461538.46153846 -1875000.         -2019230.76923077   865384.61538462]
 [-1875000.          6274038.46153846  1009615.38461538 -5769230.76923077]
 [-2019230.76923077  1009615.38461538  3461538.46153846        0.        ]
 [  865384.61538462 -5769230.76923077        0.          6274038.46153846]]
[[9.375]
 [0.   ]
 [9.375]
 [0.   ]]
[[7.11111747e-06]
 [1.11517786e-06]
 [6.53122498e-06]
 [4.46071143e-08]]


In [ ]:
#Post Processing
U=np.